# Counting Reads
The next step after mapping reads is to count the number of reads that fall within each annotated gene in the genome, so lets set up a count directory.

## Shell Variables

In [1]:
# Source the config script
source bioinf_intro_config.sh

mkdir -p $COUNT_OUT
ls $CUROUT

count_out  igv     qc_output  stuff_for_igv_shorter_intron.tgz  trimmed_fastqs
genome     myinfo  star_out   stuff_for_igv.tgz


## Counting Reads

In [2]:
htseq-count --help

usage: htseq-count [options] alignment_file gff_file

This script takes one or more alignment files in SAM/BAM format and a feature
file in GFF format and calculates for each feature the number of reads mapping
to it. See http://htseq.readthedocs.io/en/master/count.html for details.

positional arguments:
  samfilenames          Path to the SAM/BAM files containing the mapped reads.
                        If '-' is selected, read from standard input
  featuresfilename      Path to the file containing the features

optional arguments:
  -h, --help            show this help message and exit
  -f {sam,bam}, --format {sam,bam}
                        type of <alignment_file> data, either 'sam' or 'bam'
                        (default: sam)
  -r {pos,name}, --order {pos,name}
                        'pos' or 'name'. Sorting order of <alignment_file>
                        (default: name). Paired-end sequencing data must be
                        sorted either by position or by read name

We will use [htseq-count](http://www-huber.embl.de/users/anders/HTSeq/doc/count.html) to do the counting, but first we need to make some decisions, because the `htseq-count` defaults do not work with some annotation files.  Here are the **most important** commandline options that we need to consider:
* --format=<format>: Format of the input data. Possible values are sam (for text SAM files) and bam (for binary BAM files). Default is sam.
* --stranded=<yes/no/reverse>: whether the data is from a strand-specific assay (default: yes). For stranded=no, a read is considered overlapping with a feature regardless of whether it is mapped to the same or the opposite strand as the feature. For stranded=yes and single-end reads, the read has to be mapped to the same strand as the feature. For paired-end reads, the first read has to be on the same strand and the second read on the opposite strand. For stranded=reverse, these rules are reversed.
* --type=<feature type>: feature type (3rd column in GFF file) to be evaluated, all features of other type are ignored (default, suitable for RNA-Seq analysis using an Ensembl GTF file: exon)
* --idattr=<id attribute>: GFF attribute to be used as feature ID. Several GFF lines with the same feature ID will be considered as parts of the same feature. The feature ID is used to identity the counts in the output table. The default, suitable for RNA-Seq analysis using an Ensembl GTF file, is gene\_id.

And here is how we will set those options:
* --format=bam: Since Tophat generated BAM files for us
* --stranded=reverse: The dUTP method that we used for generating a strand-specific library produces reads that are anti-sense, htseq-count considers this to be "reverse".

We need to look at the GFF file to understand what exactly the `--type` and `--idattr` options are, and why we are setting them this way.

In [3]:
head -20 $GENOME_DIR/$GTF

#!genome-build CNA3
#!genome-version CNA3
#!genome-date 2015-11
#!genome-build-accession GCA_000149245.3
#!genebuild-last-updated 2015-11
1	ena	gene	100	5645	.	-	.	gene_id "CNAG_04548"; gene_source "ena"; gene_biotype "protein_coding";
1	ena	transcript	100	5645	.	-	.	gene_id "CNAG_04548"; transcript_id "AFR92135"; gene_source "ena"; gene_biotype "protein_coding"; transcript_source "ena"; transcript_biotype "protein_coding";
1	ena	exon	5494	5645	.	-	.	gene_id "CNAG_04548"; transcript_id "AFR92135"; exon_number "1"; gene_source "ena"; gene_biotype "protein_coding"; transcript_source "ena"; transcript_biotype "protein_coding"; exon_id "AFR92135-1";
1	ena	CDS	5494	5645	.	-	0	gene_id "CNAG_04548"; transcript_id "AFR92135"; exon_number "1"; gene_source "ena"; gene_biotype "protein_coding"; transcript_source "ena"; transcript_biotype "protein_coding"; protein_id "AFR92135"; protein_version "1";
1	ena	start_codon	5643	5645	.	-	0	gene_id "CNAG_04548"; transcript_id "AFR92135"; exon_number "1"; 

In [4]:
head -50 $GENOME_DIR/$GTF | cut -c -55

#!genome-build CNA3
#!genome-version CNA3
#!genome-date 2015-11
#!genome-build-accession GCA_000149245.3
#!genebuild-last-updated 2015-11
1	ena	gene	100	5645	.	-	.	gene_id "CNAG_04548"; gene_so
1	ena	transcript	100	5645	.	-	.	gene_id "CNAG_04548"; t
1	ena	exon	5494	5645	.	-	.	gene_id "CNAG_04548"; transc
1	ena	CDS	5494	5645	.	-	0	gene_id "CNAG_04548"; transcr
1	ena	start_codon	5643	5645	.	-	0	gene_id "CNAG_04548";
1	ena	exon	5322	5422	.	-	.	gene_id "CNAG_04548"; transc
1	ena	CDS	5322	5422	.	-	1	gene_id "CNAG_04548"; transcr
1	ena	exon	3958	5263	.	-	.	gene_id "CNAG_04548"; transc
1	ena	CDS	3958	5263	.	-	2	gene_id "CNAG_04548"; transcr
1	ena	exon	3206	3890	.	-	.	gene_id "CNAG_04548"; transc
1	ena	CDS	3206	3890	.	-	1	gene_id "CNAG_04548"; transcr
1	ena	exon	2846	3126	.	-	.	gene_id "CNAG_04548"; transc
1	ena	CDS	2846	3126	.	-	0	gene_id "CNAG_04548"; transcr
1	ena	exon	2322	2782	.	-	.	gene_id "CNAG_04548"; transc
1	ena	CDS	2322	2782	.	-	1	gene_id "CNAG_04548"; transcr
1	ena	exon	1823	2274	.

## Running htseq-count
So now we are ready!  We run `htseq-count` using `htseq-count ALIGNMENT_FILE GFF_FILE`.  Here is our command for our test sample:
    
* --format=bam: Since Tophat generated BAM files for us
* --stranded=reverse: The dUTP method that we used for generating a strand-specific library produces reads that are anti-sense, htseq-count considers this to be "reverse".


In [5]:
ls ${STAR_OUT}

21_2019_P_M1_S21_L001_R1_short_introns_Aligned.sortedByCoord.out.bam
21_2019_P_M1_S21_L001_R1_short_introns_Aligned.sortedByCoord.out.bam.bai
21_2019_P_M1_S21_L001_R1_short_introns_Log.final.out
21_2019_P_M1_S21_L001_R1_short_introns_Log.out
21_2019_P_M1_S21_L001_R1_short_introns_Log.progress.out
21_2019_P_M1_S21_L001_R1_short_introns_ReadsPerGene.out.tab
21_2019_P_M1_S21_L001_R1_short_introns_SJ.out.tab
21_2019_P_M1_S21_L002_R1_Aligned.out.bam
21_2019_P_M1_S21_L002_R1_Log.final.out
21_2019_P_M1_S21_L002_R1_Log.out
21_2019_P_M1_S21_L002_R1_Log.progress.out
21_2019_P_M1_S21_L002_R1_ReadsPerGene.out.tab
21_2019_P_M1_S21_L002_R1_short_introns_Aligned.sortedByCoord.out.bam
21_2019_P_M1_S21_L002_R1_short_introns_Aligned.sortedByCoord.out.bam.bai
21_2019_P_M1_S21_L002_R1_short_introns_Log.final.out
21_2019_P_M1_S21_L002_R1_short_introns_Log.out
21_2019_P_M1_S21_L002_R1_short_introns_Log.progress.out
21_2019_P_M1_S21_L002_R1_short_introns_ReadsPerGene.out.tab
21_2019_P_M1_S21_L002_R1_short_in

In [6]:
htseq-count --quiet \
    --format=bam \
    --stranded=reverse \
    ${STAR_OUT}/21_2019_P_M1_S21_L002_R1_Aligned.out.bam \
    $GENOME_DIR/$GTF > ${COUNT_OUT}/21_2019_P_M1_S21_L002_R1.tsv

Let's take a quick peek at the results

In [7]:
head ${COUNT_OUT}/21_2019_P_M1_S21_L002_R1.tsv

CNAG_00001	0
CNAG_00002	51
CNAG_00003	23
CNAG_00004	78
CNAG_00005	5
CNAG_00006	546
CNAG_00007	188
CNAG_00008	119
CNAG_00009	29
CNAG_00010	146


There's also some useful information at the end of the file:

In [8]:
tail ${COUNT_OUT}/21_2019_P_M1_S21_L002_R1.tsv

ENSRNA049551862	0
ENSRNA049551899	0
ENSRNA049551942	0
ENSRNA049551964	0
ENSRNA049551993	0
__no_feature	18783
__ambiguous	316
__too_low_aQual	0
__not_aligned	46060
__alignment_not_unique	35006
